<a href="https://colab.research.google.com/github/Vaelastraszz/Images_Classification_Tensorflow/blob/master/Projet_7_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


<h2> Loading des données training/validation/test set <h2>

<h3> Loading des données </h3>

In [0]:
import os

def get_images() :
    
    img_list = []
    start_path = '/content/gdrive/My Drive/Dogs/Images/' # current directory
    
    for path,dirs,files in os.walk(start_path):
        for filename in files:
            img_list.append(os.path.join(path,filename))
    
    return img_list

In [0]:
import re

def get_labels(list_img):
    
    labels = []
    
    for i in range(len(list_img)):
    
        m = re.search('/content/gdrive/My Drive/Dogs/Images/(.+?)/', list_img[i])
        if m:
            found = m.group(1)
    
        labels.append(found[10:])
    
    return labels

In [0]:
full_list = get_images()
full_labels = get_labels(full_list)

In [0]:
full_labels

<h3> Splitting des set </h3>

In [0]:
from sklearn import model_selection

In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(full_list, full_labels, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [0]:
np.array(X_train)

array(['/content/gdrive/My Drive/Dogs/Images/n02099712-Labrador_retriever/n02099712_5021.jpg',
       '/content/gdrive/My Drive/Dogs/Images/n02086646-Blenheim_spaniel/n02086646_946.jpg',
       '/content/gdrive/My Drive/Dogs/Images/n02094433-Yorkshire_terrier/n02094433_2596.jpg',
       ...,
       '/content/gdrive/My Drive/Dogs/Images/n02097474-Tibetan_terrier/n02097474_7766.jpg',
       '/content/gdrive/My Drive/Dogs/Images/n02102040-English_springer/n02102040_251.jpg',
       '/content/gdrive/My Drive/Dogs/Images/n02099601-golden_retriever/n02099601_7304.jpg'],
      dtype='<U97')

<h3> Préparation en dataframes pour le datagen Keras </h3> 

In [0]:
train_df = pd.DataFrame({'path': X_train,'label':y_train})

In [0]:
val_df = pd.DataFrame({'path': X_val,'label':y_val})

In [0]:
test_df = pd.DataFrame({'path': X_test,'label':y_test})

In [0]:
train_df.head()

,path,label
0,/content/gdrive/My Drive/Dogs/Images/n02099712...,Labrador_retriever
1,/content/gdrive/My Drive/Dogs/Images/n02086646...,Blenheim_spaniel
2,/content/gdrive/My Drive/Dogs/Images/n02094433...,Yorkshire_terrier
3,/content/gdrive/My Drive/Dogs/Images/n02109525...,Saint_Bernard
4,/content/gdrive/My Drive/Dogs/Images/n02096177...,cairn


<h2> Récupération de l'architecture entrainée et fine-tuning </h2>

In [0]:
from keras import applications
from keras.regularizers import l2

In [0]:
#vgg = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
xcp = applications.xception.Xception(weights='imagenet', include_top=False)


In [0]:
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(xcp)
 
# Add new layers
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dropout(rate = 0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(120, activation='softmax'))

In [0]:
for layer in xcp.layers:
    layer.trainable = False

In [0]:
for layer in xcp.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fa3d6b7a9b0> False
<keras.layers.convolutional.Conv2D object at 0x7fa3d6b7aeb8> False
<keras.layers.normalization.BatchNormalization object at 0x7fa3d6ba1c50> False
<keras.layers.core.Activation object at 0x7fa3d6b7ad68> False
<keras.layers.convolutional.Conv2D object at 0x7fa3d6b48c18> False
<keras.layers.normalization.BatchNormalization object at 0x7fa3e2eea9b0> False
<keras.layers.core.Activation object at 0x7fa3e2e868d0> False
<keras.layers.convolutional.SeparableConv2D object at 0x7fa3e2d9bf60> False
<keras.layers.normalization.BatchNormalization object at 0x7fa3e2d35c50> False
<keras.layers.core.Activation object at 0x7fa3e2c5a6d8> False
<keras.layers.convolutional.SeparableConv2D object at 0x7fa3e2c1eb70> False
<keras.layers.normalization.BatchNormalization object at 0x7fa3e2bf0a20> False
<keras.layers.convolutional.Conv2D object at 0x7fa3e2e196a0> False
<keras.layers.pooling.MaxPooling2D object at 0x7fa3e2b5bf60> False
<keras.la

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
batch_normalization_9 (Batch (None, None, None, 2048)  8192      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 120)               123000    
Total para

<h2> Data Augmentation sur les split set </h2>

In [0]:
train_batch_size = 32
val_batch_size = 16
img_width, img_height = 299, 299

In [0]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col ="path",
    y_col ="label",
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=train_batch_size,
    class_mode='categorical')

Found 12348 validated image filenames belonging to 120 classes.


In [0]:
validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col ="path",
    y_col ="label",
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=val_batch_size,
    class_mode='categorical')

Found 4116 validated image filenames belonging to 120 classes.


In [0]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col ="path",
    y_col ="label",
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

Found 4116 validated image filenames belonging to 120 classes.


<h2> Model Training </h2>

In [0]:
import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint
 
#epochs = 1
#learning_rate = 0.001
 
# checkpoints
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))
 
bst_model_path = "{}.h5".format(STAMP)
model_checkpoint = ModelCheckpoint(bst_model_path,
save_best_only=True,
save_weights_only=False,
verbose=1)

In [0]:
from keras import optimizers

In [0]:
model.compile(loss='categorical_crossentropy',
               optimizer=optimizers.Adam(lr=0.001), metrics=["accuracy"])

In [0]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint],
    validation_steps=validation_generator.batch_size // validation_generator.batch_size)

Epoch 1/10
385/385 [==============================] - 389s 1s/step - loss: 1.4561 - acc: 0.6410 - val_loss: 0.1184 - val_acc: 0.9375

Epoch 00001: val_loss improved from inf to 0.11843, saving model to 2019-05-23_dog_breed_model.h5
Epoch 2/10
385/385 [==============================] - 367s 954ms/step - loss: 1.0883 - acc: 0.7229 - val_loss: 1.6733 - val_acc: 0.6250

Epoch 00002: val_loss did not improve from 0.11843
Epoch 3/10
385/385 [==============================] - 375s 974ms/step - loss: 1.0024 - acc: 0.7365 - val_loss: 0.5508 - val_acc: 0.8125

Epoch 00003: val_loss did not improve from 0.11843
Epoch 4/10
385/385 [==============================] - 374s 971ms/step - loss: 0.9074 - acc: 0.7552 - val_loss: 0.0962 - val_acc: 0.9375

Epoch 00004: val_loss improved from 0.11843 to 0.09616, saving model to 2019-05-23_dog_breed_model.h5
Epoch 5/10
385/385 [==============================] - 374s 971ms/step - loss: 0.8541 - acc: 0.7657 - val_loss: 0.5165 - val_acc: 0.7500

Epoch 00005: val

<h3> Save model </h3>

In [0]:
model.save('/content/gdrive/My Drive/Dogs/dogs_cnn.h5')

<h3> Load model </h3>

In [0]:
model = load_model('/content/gdrive/My Drive/Dogs/dogs_cnn.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


<h2> Prédiction <h2>

In [0]:
from keras.models import load_model
from keras.preprocessing import image

In [0]:
label_map = (train_generator.class_indices)

<h3> Serialize my labels for using in a portative script </h3>

In [0]:
import pickle

pickle.dump(label_map, open('/content/gdrive/My Drive/Dogs/labels.pkl','wb'))

<h3> Script </h3>

In [0]:
inverted_lab = dict([[v,k] for k,v in label_map.items()])


In [0]:
def predict_from_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_tensor = image.img_to_array(img) # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0) # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.
 
    pred = model.predict(img_tensor)
    predicted_class = inverted_lab[np.argmax(pred)]
 
    return predicted_class

In [0]:
predict_from_image("/content/gdrive/My Drive/Dogs/Test/kikou.jpg")

'Labrador_retriever'

In [0]:
label_map

<h3> Test set prediction </h3>

In [0]:
prediction = model.predict_generator(test_generator, 
                                     steps=len(test_generator), 
                                     max_queue_size=10, 
                                     workers=1, 
                                     use_multiprocessing=False, 
                                     verbose=1)

4116/4116 [==============================] - 54s 13ms/step


In [0]:
prediction.shape

(4116, 120)

In [0]:
np.argmax(prediction[6])

109

In [0]:
y_true = test_generator.classes

In [0]:
total = 0
for i in range(len(y_true)):
    
    if y_true[i]==np.argmax(prediction[i]):
        total+=1
        
print("Pourcentage de réussite sur un test set inconnu au modèle: ",np.round(total/len(y_true)*100),"%")

Pourcentage de réussite sur un test set inconnu au modèle:  85.0 %
